<a href="https://colab.research.google.com/github/vdovych/nlp_js_project/blob/master/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### TODO:
__need__:
* balance classes ?

__would be good__

__experiment__

__questions__
* як лейбити 

### data mixing & preprocessing

In [0]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV

In [0]:
gen_target = pd.read_csv('bbc_target.csv', header=None)[0].append(pd.Series(data=fetch_20newsgroups(subset='all').target), ignore_index=True)
gen_text = pd.read_csv('bbc_text.csv', header=None)[0].append(pd.Series(data=fetch_20newsgroups(subset='all', remove=('headers', 'footers')).data), ignore_index=True)
gen_target = gen_target.append(pd.read_csv('webhouse_target.csv', header=None)[0])
gen_text = gen_text.append(pd.read_csv('webhouse_text.csv', header=None)[0])

In [101]:
gen_target.shape, gen_text.shape

((111682,), (111682,))

In [0]:
gen_target.replace({"'business'":20, "'entertainment'":21, "'politics'":22, "'sport'":23, "'tech'":24, "'finance'":25, "'travel'":26, "'sports'":23 }, inplace=True)

In [103]:
unique, counts = np.unique(gen_target, return_counts=True)
dict(zip(unique, counts))

{0: 799,
 1: 973,
 2: 985,
 3: 982,
 4: 963,
 5: 988,
 6: 975,
 7: 990,
 8: 996,
 9: 994,
 10: 999,
 11: 991,
 12: 984,
 13: 990,
 14: 987,
 15: 997,
 16: 910,
 17: 940,
 18: 775,
 19: 628,
 20: 510,
 21: 10454,
 22: 10485,
 23: 10578,
 24: 40673,
 25: 10068,
 26: 10068}

In [0]:
x_train, x_test, y_train, y_test = train_test_split(gen_text, gen_target, stratify=gen_target)

In [0]:
# tfidf = TfidfVectorizer(sublinear_tf=True, norm='l2', stop_words='english', min_df=20, max_df=100000, ngram_range=(1, 2))
tfidf = TfidfVectorizer(stop_words='english', norm=None, min_df=0.0003, ngram_range=(1, 2))
features = tfidf.fit_transform(x_train)

In [106]:
features.shape

(83761, 70371)

In [0]:
# ['alt.atheism',0,0
#  'comp.graphics',1,1
#  'comp.os.ms-windows.misc',2,1
#  'comp.sys.ibm.pc.hardware',3,1
#  'comp.sys.mac.hardware',4,1
#  'comp.windows.x',5,1
#  'misc.forsale',6,1
#  'rec.autos',7,0
#  'rec.motorcycles',8,0
#  'rec.sport.baseball',9.0
#  'rec.sport.hockey',10,0
#  'sci.crypt',11,1
#  'sci.electronics',12,1
#  'sci.med',13,1
#  'sci.space',14,1
#  'soc.religion.christian',15,0
#  'talk.politics.guns',16,0
#  'talk.politics.mideast',17,0
#  'talk.politics.misc',18,0
#  'talk.religion.misc',19,0]
#  'business',20, 1
#  'entertainment',21, 0
#  'politics',22, 0
#  'sport',23, 0
#  'tech',24, 1
#  'finance',25,0
#  'travel', 26, 0
tech = [1,2,3,4,5,6,11,12,13,14,20,24]

In [0]:
bin_tr_y = np.isin(y_train.values, tech).astype(np.uint8)
bin_tes_y = np.isin(y_test.values, tech).astype(np.uint8)

In [0]:
vectors_test = tfidf.transform(x_test)

In [110]:
# np.savetxt('test_x', vectors_test.toarray()[:2])
clf = MultinomialNB(alpha=.01)
clf.fit(features, bin_tr_y)
pred = clf.predict(vectors_test)
accuracy_score(bin_tes_y, pred)

0.8656566741878873

In [111]:
unique, counts = np.unique(bin_tr_y, return_counts=True)
dict(zip(unique, counts))

{0: 45509, 1: 38252}

### Model & train

In [0]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
import keras

In [0]:
simple_model = Sequential([Dense(150,activation='relu', input_dim=features.shape[1], kernel_regularizer=l2(0.01)),
                           Dropout(0.6, noise_shape=None, seed=None),
                           Dense(200,activation='relu', kernel_regularizer=l2(0.01)),
                           Dropout(0.4, noise_shape=None, seed=None),
                           Dense(1,activation='sigmoid')])
ES_callback = EarlyStopping(monitor='val_binary_accuracy', min_delta=0.00005, patience=4)

filepath="improvement-{epoch:02d}-{loss:.4f}-{val_binary_accuracy:.4f}.hdf5"
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_binary_accuracy', verbose=1, save_best_only=True, mode='max')

In [0]:
simple_model.compile(loss='binary_crossentropy',
                     optimizer='adam',
                     metrics=['binary_accuracy'])

In [128]:
simple_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_43 (Dense)             (None, 150)               10555800  
_________________________________________________________________
dropout_29 (Dropout)         (None, 150)               0         
_________________________________________________________________
dense_44 (Dense)             (None, 200)               30200     
_________________________________________________________________
dropout_30 (Dropout)         (None, 200)               0         
_________________________________________________________________
dense_45 (Dense)             (None, 1)                 201       
Total params: 10,586,201
Trainable params: 10,586,201
Non-trainable params: 0
_________________________________________________________________


In [129]:
model_output= simple_model.fit(features,
                               bin_tr_y,
                               batch_size=512,
                               epochs=40,
                               verbose=1,
                               validation_data=(vectors_test, bin_tes_y),
#                                callbacks=[ES_callback])
                               callbacks=[ES_callback, checkpoint])

Train on 83761 samples, validate on 27921 samples
Epoch 1/40
83761/83761 [==============================] - 46s 550us/step - loss: 1.5588 - binary_accuracy: 0.8682 - val_loss: 1.0029 - val_binary_accuracy: 0.8958

Epoch 00001: val_binary_accuracy improved from -inf to 0.89585, saving model to improvement-01-1.5588-0.8958.hdf5
Epoch 2/40
83761/83761 [==============================] - 44s 530us/step - loss: 0.9712 - binary_accuracy: 0.8922 - val_loss: 0.9283 - val_binary_accuracy: 0.9005

Epoch 00002: val_binary_accuracy improved from 0.89585 to 0.90050, saving model to improvement-02-0.9712-0.9005.hdf5
Epoch 3/40
83761/83761 [==============================] - 44s 528us/step - loss: 0.9652 - binary_accuracy: 0.8935 - val_loss: 0.9705 - val_binary_accuracy: 0.8965

Epoch 00003: val_binary_accuracy did not improve from 0.90050
Epoch 4/40
83761/83761 [==============================] - 45s 532us/step - loss: 0.9973 - binary_accuracy: 0.8935 - val_loss: 1.0017 - val_binary_accuracy: 0.8998

E

In [0]:
simple_model = Sequential([Dense(150,activation='relu', input_dim=70371, kernel_regularizer=l2(0.01)),
                           Dropout(0.6, noise_shape=None, seed=None),
                           Dense(200,activation='relu', kernel_regularizer=l2(0.01)),
                           Dropout(0.4, noise_shape=None, seed=None),
                           Dense(1,activation='sigmoid')])

simple_model.load_weights('improvement-05-1.0001-0.9013.hdf5')

simple_model.compile(loss='binary_crossentropy',
                     optimizer='adam',
                     metrics=['binary_accuracy'])

#### Model evaluation


In [0]:
predicted = simple_model.predict_classes(vectors_test)

In [132]:
confusion_matrix(bin_tes_y, predicted)

array([[13691,  1481],
       [ 1274, 11475]])

In [133]:
print(classification_report(bin_tes_y, predicted))

              precision    recall  f1-score   support

           0       0.91      0.90      0.91     15172
           1       0.89      0.90      0.89     12749

   micro avg       0.90      0.90      0.90     27921
   macro avg       0.90      0.90      0.90     27921
weighted avg       0.90      0.90      0.90     27921



#### Grid search

vectorizer
* min_df 0.0001 4
* max_df 0.15 ...

model
* layers
* batch_size    512, 256
* epochs        20, 40, 
* dense 100-300
* dropout




In [0]:
# cut data
la_x_train, la_x_test, la_y_train, la_y_test = train_test_split(x_test, y_test, stratify=y_test)
la_x_train, la_x_test, la_y_train, la_y_test = train_test_split(la_x_test, la_y_test, stratify=la_y_test)
la_x_train, la_x_test, la_y_train, la_y_test = train_test_split(la_x_train, la_y_train, stratify=la_y_train)
la_x_train, la_x_test, la_y_train, la_y_test = train_test_split(la_x_train, la_y_train, stratify=la_y_train)
gr_x_train, gr_x_test, gr_y_train, gr_y_test = train_test_split(la_x_train, la_y_train, stratify=la_y_train)

In [0]:
gr_y_train = np.isin(gr_y_train.values, tech).astype(np.uint8)
gr_y_test = np.isin(gr_y_test.values, tech).astype(np.uint8)

In [0]:
from keras.wrappers.scikit_learn import KerasClassifier

ES_callback = EarlyStopping(monitor='loss', min_delta=0.00005, patience=4)

def create_model(neurons=100,neurons2=100, dropout=0.5,dropout2=0.5, layers=1):
    
    model = Sequential()
    model.add(Dense(neurons,activation='relu', input_dim=features.shape[1], kernel_regularizer=l2(0.01)))
    model.add(Dropout(dropout, noise_shape=None, seed=None))
    if layers>=2:                   
        model.add(Dense(neurons2,activation='relu', kernel_regularizer=l2(0.01)))
        model.add(Dropout(dropout2, noise_shape=None, seed=None))

    model.add(Dense(1,activation='sigmoid'))

    model.compile(loss='binary_crossentropy',
                         optimizer='adam',
                         metrics=['accuracy'])
    return model


model = KerasClassifier(build_fn=create_model, verbose=1)

In [0]:
# define the grid search parameters
neurons = [100, 150]
layers = [1,2]
batch_size = [512]
dropout = [0.6]
# batch_size = [128, 256, 512]
# dropout = [0.35, 0.45, 0.6]

callbacks = [[ES_callback]]
validation_data=[(tfidf.transform(gr_x_test), gr_y_test)]
epochs = [10]

param_grid = dict(neurons=neurons,
                  neurons2=neurons,
                  dropout=dropout,
                  dropout2=dropout, 
                  layers=layers,
                 batch_size=batch_size,
                 callbacks=callbacks,
                  validation_data=validation_data,
                 epochs=epochs)

grid = GridSearchCV(estimator=model,
                    param_grid=param_grid,
#                     n_jobs=-1,
                   verbose=1)

grid_result = grid.fit(tfidf.transform(gr_x_train), gr_y_train)

In [0]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


Best: 0.763134 using {'batch_size': 512, 'callbacks': [<keras.callbacks.EarlyStopping object at 0x7f602a096f98>], 'dropout': 0.6, 'dropout2': 0.6, 'epochs': 10, 'layers': 2, 'neurons': 100, 'neurons2': 150, 'validation_data': (<736x92743 sparse matrix of type '<class 'numpy.float64'>'
	with 23265 stored elements in Compressed Sparse Row format>, array([0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1,
       1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1,
       0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1

#### Saving the most successfully weighted model

In [140]:
!pip install tensorflowjs

In [0]:
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(simple_model, 'model')

#### tfidf ctreation from file

In [0]:
tfidf = TfidfVectorizer(stop_words='english', vocabulary=vocab, norm=None, min_df=0.0003, ngram_range=(1, 2))

In [0]:
tfidf.idf_=idf

In [0]:
text = """In the beginning was the Word, and the Word was with God, and the Word was God.
2The same was in the beginning with God.
3All things were made by him; and without him was not any thing made that was made.
4In him was life; and the life was the light of men.
5And the light shineth in darkness; and the darkness comprehended it not.
6There was a man sent from God, whose name was John.
7The same came for a witness, to bear witness of the Light, that all men through him might believe.
8He was not that Light, but was sent to bear witness of that Light.
9That was the true Light, which lighteth every man that cometh into the world.
10He was in the world, and the world was made by him, and the world knew him not.
11He came unto his own, and his own received him not.
12But as many as received him, to them gave he power to become the sons of God, even to them that believe on his name:
13Which were born, not of blood, nor of the will of the flesh, nor of the will of man, but of God.
14And the Word was made flesh, and dwelt among us, (and we beheld his glory, the glory as of the only begotten of the Father,) full of grace and truth.
15John bare witness of him, and cried, saying, This was he of whom I spake, He that cometh after me is preferred before me: for he was before me.
16And of his fulness have all we received, and grace for grace.
17For the law was given by Moses, but grace and truth came by Jesus Christ.
18No man hath seen God at any time; the only begotten Son, which is in the bosom of the Father, he hath declared him.
19And this is the record of John, when the Jews sent priests and Levites from Jerusalem to ask him, Who art thou?
20And he confessed, and denied not; but confessed, I am not the Christ.
21And they asked him, What then? Art thou Elias? And he saith, I am not. Art thou that prophet? And he answered, No.
22Then said they unto him, Who art thou? that we may give an answer to them that sent us. What sayest thou of thyself?
23He said, I am the voice of one crying in the wilderness, Make straight the way of the Lord, as said the prophet Esaias.
24And they which were sent were of the Pharisees.
25And they asked him, and said unto him, Why baptizest thou then, if thou be not that Christ, nor Elias, neither that prophet?
26John answered them, saying, I baptize with water: but there standeth one among you, whom ye know not;
27He it is, who coming after me is preferred before me, whose shoe's latchet I am not worthy to unloose.
28These things were done in Bethabara beyond Jordan, where John was baptizing.
29The next day John seeth Jesus coming unto him, and saith, Behold the Lamb of God, which taketh away the sin of the world.
30This is he of whom I said, After me cometh a man which is preferred before me: for he was before me.
31And I knew him not: but that he should be made manifest to Israel, therefore am I come baptizing with water.
32And John bare record, saying, I saw the Spirit descending from heaven like a dove, and it abode upon him.
33And I knew him not: but he that sent me to baptize with water, the same said unto me, Upon whom thou shalt see the Spirit descending, and remaining on him, the same is he which baptizeth with the Holy Ghost.
34And I saw, and bare record that this is the Son of God.
35Again the next day after John stood, and two of his disciples;
36And looking upon Jesus as he walked, he saith, Behold the Lamb of God!
37And the two disciples heard him speak, and they followed Jesus.
38Then Jesus turned, and saw them following, and saith unto them, What seek ye? They said unto him, Rabbi, (which is to say, being interpreted, Master,) where dwellest thou?
39He saith unto them, Come and see. They came and saw where he dwelt, and abode with him that day: for it was about the tenth hour.
40One of the two which heard John speak, and followed him, was Andrew, Simon Peter's brother."""

In [0]:
transformed = tfidf.transform([text])

In [39]:
simple_model.predict(transformed)

array([[0.01392901]], dtype=float32)

#### Manual trial

In [0]:
# text = 'i want to use this classifier for a commercial project. It is light-weighted and can work in a chrome plugin'
# text = 'i care for my country very much, vote for me. believeme, i will be the best president you have ever known'
# text = 'This ballet style is often performed barefoot. Contemporary ballets may include mime and acting, and are usually set to music (typically orchestral but occasionally vocal). It can be difficult to differentiate this form from neoclassical or modern ballet. Contemporary ballet is also close to contemporary dance, because many contemporary ballet concepts come from the ideas and innovations of twentieth-century modern dance, including floor work and turn-in of the legs. The main distinction is that ballet technique is essential to perform a contemporary ballet.George Balanchine is considered to have been a pioneer of contemporary ballet. Another early contemporary ballet choreographer, Twyla Tharp, choreographed Push Comes To Shove for the American Ballet Theatre in 1976, and in 1986 created In The Upper Room for her own company. Both of these pieces were considered innovative for their melding of distinctly modern movements with the use of pointe shoes and classically trained dancers.Today there are many contemporary ballet companies and choreographers. These include Alonzo King and his company LINES Ballet; Matthew Bourne and his company New Adventures; Complexions Contemporary Ballet; Nacho Duato and his Compañia Nacional de Danza; William Forsythe and The Forsythe Company; and Jiří Kylián of the Nederlands Dans Theater. Traditionally "classical" companies, such as the Mariinsky (Kirov) Ballet and the Paris Opera Ballet, also regularly perform contemporary works.The term ballet has evolved to include all forms associated with it. Someone training as a ballet dancer will now be expected to perform neoclassical, modern and contemporary work. A ballet dancer is expected to be able to be stately and regal for classical work, free and lyrical in neoclassical work, and unassuming, harsh or pedestrian for modern and contemporary work. In addition, there are several modern varieties of dance that fuse classical ballet technique with contemporary dance, such as Hiplet, that require dancers to be practised in non-Western dance styles.'
# text='The first thing that comes into our minds while thinking of employees at tech companies is probably Software Engineers and Data Scientists. But a technology company requires employees beyond those in technical roles, so they hire for non-tech roles too. The current scenario at the job market doesn’t necessarily require you to be a tech pro to get hired by technology companies. As the growth tech companies increases, they need more non-technical workforce to help transform their technology into revenue. Roles such as sales, marketing, business operation, etc. are crucial to their expansion. These highly coveted positions are also well compensated by tech employers.'
# # text = '''As part of an effort to reduce noise in Ghana’s capital, Accra, the environment minister has suggested that the Muslim call to prayer, normally broadcast over loudspeakers across the city, should instead be sent out on WhatsApp. The notion has proved immensely unpopular – not least because it equates the call to prayer with noise pollution. But it also highlights religion’s growing, if sometimes uneasy, reliance on tech.

# Contactless collection Catholic and Protestant churches in the UK have begun using contactless card readers for donations and other payments, hoping to make life easier for parishioners who may not be carrying cash. A contactless collection plate is being trialled by the Church of England, but it is being held up because it is feared the technology might slow things down.

# The confession app Confession (version 2.1) walks sinners through the business of confession, pings you push notifications when it’s time for your next shriving and includes a handy sin checklist in case you have forgotten what you’ve done wrong. What it doesn’t do is offer absolution. You still need a priest for that.

# Televangelist tech American TV preachers tend to be at home in the digital realm, particularly those espousing “prosperity theology”, which holds that God rewards the pious with material wealth. It works for them, anyway. Controversial televangelist Joel Osteen’s free app offers daily devotional messages and sermons. And Creflo Dollar’s latest app boasts a “give” feature that makes handing your money over to Creflo A Dollar Ministries as easy as gambling. Webcast rites Funerals broadcast online are at least a decade old. Many UK crematoria now offer it as part of the package, and rabbis and vicars routinely officiate at live-streamed services. It is not quite the same as being there, but in many cases it’s probably preferable.Halal apps There are several apps that help Muslims find halal restaurants and stores in the immediate area . There is even one – Scan Halal – that allows you to scan the barcode of any food in a supermarket to instantly determine whether it contains non-halal ingredients. Online reviews for this idea are decidedly mixed.'
# '''
# text = "Redux Redux DevTools debugging"
# text = "Redux DevTools for debugging application's state changes."
text = "NodeSource is a company focused on providing enterprise-grade Node support and they maintain a repository containing the latest versions of Node.js."
text = """made that was made.
4In him was life; and the life was the light of men.
5And the light shineth in darkness; and the darkness comprehended it not.
6There was a man sent from God, whose name was John.
7The same came for a witness, to bear witness of the Light, that all men through him might believe.
8He was not that Light, but was sent to bear witness of that Light.
9That was the true Light, which lighteth every man that cometh into the world."""

In [0]:
transformed = tfidf.transform([text])

In [0]:
tfidf._tfidf._idf_diag.toarray()

In [0]:
transformed.toarray().tolist()

In [0]:
tfidf.idf_.tolist()

In [0]:
tfidf.vocabulary_

In [0]:
simple_model.predict(transformed)


array([[0.49849725]], dtype=float32)

#### for js code testing

In [0]:
text = ["Redux DevTools for debugging application's state changes.", "I want to use this classifier for a commercial project.", "for debugging application's I want to use this classifier"]

In [0]:
tfidf = TfidfVectorizer(stop_words='english', norm=None, use_idf=True, smooth_idf=True, sublinear_tf=True)
feat = tfidf.fit_transform(text)

In [0]:
simple_model = Sequential([Dense(100,activation='relu', input_dim=11, kernel_regularizer=l2(0.01)),
                           Dropout(0.5, noise_shape=None, seed=None),
                           Dense(100,activation='relu', kernel_regularizer=l2(0.01)),
                           Dropout(0.3, noise_shape=None, seed=None),
                           Dense(1,activation='sigmoid')])

ES_callback = EarlyStopping(monitor='val_loss', min_delta=0.00005, patience=4)

# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}-{acc:.4f}.hdf5"
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')

In [0]:
simple_model.compile(loss='binary_crossentropy',
                     optimizer='adam',
                     metrics=['accuracy'])

In [0]:
model_output= simple_model.fit(feat,
                               [1,0,0],
                               verbose=1,
                               callbacks=[ES_callback, checkpoint])

Epoch 1/1
3/3 [==============================] - 2s 778ms/step - loss: 1.8090 - acc: 0.6667

Epoch 00001: loss improved from inf to 1.80903, saving model to weights-improvement-01-1.8090-0.6667.hdf5


/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


#### Save data for further tf-idf vectorization

In [0]:
import json

In [0]:
def default(o):
    if isinstance(o, np.int64): return int(o)  
    if isinstance(o, np.float64): return float(o)
    raise TypeError

In [0]:
with open('tfidf_vocab.json', 'w') as f:
    json.dump(tfidf.vocabulary_, f, default=default)

In [0]:
with open('idf_.json', 'w') as f:
    json.dump(tfidf.idf_.tolist(), f)

In [0]:
# with open('transformed.json', 'w') as f:
#     json.dump(transformed.toarray().tolist(), f)

In [0]:
with open('idf_.json') as f:
    idf=json.load(f)
idf

[4.6324736418843475, 3.958650734150805, 8.270059801610733, 7.6722228008551125, 8.470730497072886, 7.895366352169322, 8.183048424621104, 8.470730497072886, 5.442208400695902, 5.316096940933037, 8.270059801610733, 8.183048424621104, 5.726312651799799, 7.777583316512939, 8.470730497072886, 5.592781259175277, 7.11260701291969, 8.103005716947568, 5.624529957489857, 8.270059801610733, 6.225303817918787, 8.270059801610733, 6.114078182808563, 8.470730497072886, 6.093244095905721, 8.470730497072886, 8.470730497072886, 8.470730497072886, 5.995126239947197, 8.365369981415057, 8.365369981415057, 8.028897744793845, 8.183048424621104, 8.365369981415057, 8.028897744793845, 8.183048424621104, 8.103005716947568, 8.470730497072886, 8.103005716947568, 7.4899012440611585, 8.183048424621104, 8.365369981415057, 7.959904873306893, 8.365369981415057, 8.365369981415057, 7.895366352169322, 8.470730497072886, 8.365369981415057, 8.183048424621104, 8.470730497072886, 8.103005716947568, 3.386569410838821, 4.1254831

In [13]:
with open('tfidf_vocab.json') as f:
    vocab=json.load(f)
type(vocab)

dict

In [12]:
with open('idf_.json') as f:
    idf=json.load(f)
idf

[4.107290840001257,
 7.460537399804139,
 9.003530212830087,
 8.443914424894665,
 9.003530212830087,
 8.869998820205563,
 8.724816810361066,
 8.934537341343134,
 8.116227017829184,
 8.073054845963975,
 7.115378897926965,
 7.372890092745383,
 7.565050098539626,
 7.325099428909034,
 8.869998820205563,
 8.901747518520144,
 9.03989785700096,
 9.003530212830087,
 8.80937419838913,
 5.793262762498486,
 8.83922716153881,
 8.598065104721922,
 9.03989785700096,
 5.239841501907759,
 8.32840153777282,
 8.672173076875644,
 7.515453157400254,
 8.724816810361066,
 9.003530212830087,
 7.991929301151607,
 8.780386661515877,
 8.423711717577145,
 7.720614206164031,
 7.626204521692956,
 8.52907223323497,
 7.445385594783537,
 4.815500166530663,
 8.258197279099571,
 8.968438893018817,
 8.780386661515877,
 8.29268345517074,
 6.622001917495921,
 8.934537341343134,
 7.437894923054379,
 8.52907223323497,
 8.724816810361066,
 8.310383032270142,
 5.017858524378795,
 8.901747518520144,
 8.32840153777282,
 6.622001